In [ ]:
val tripRaw = sc.textFile("data/tripdata.csv")
val taxiRaw = sc.textFile("data/taxi_zone_lookup.csv")

val tripHeader = tripRaw.first()
val taxiHeader = taxiRaw.first()

val trip = tripRaw.filter(_ != tripHeader)
val taxi = taxiRaw.filter(_ != taxiHeader)

In [ ]:

val tripParsed = trip.map(line => line.split(","))

In [ ]:

val tripParsed = trip.map { line =>
  val arr = line.split(",")
  try {
    val hour = arr(1).substring(11, 13).toInt
    val puLocationId = arr(7).toInt
    (puLocationId, hour)
  } catch {
    case _: Throwable => (-1, -1)
  }
}.filter { case (pu, h) => pu != -1 && h != -1 }

tripParsed.count()

In [ ]:
val taxiParsed = taxi.map { line =>
  val arr = line.split(",")
  val locationId = arr(0).toInt
  val borough = arr(1)
  (locationId, borough)
}

taxiParsed.count()

In [ ]:
val joined = tripParsed.join(taxiParsed)

val ordersByBoroughHour = joined
  .map { case (_, (hour, borough)) => ((borough, hour), 1) }
  .reduceByKey(_ + _)

val result = ordersByBoroughHour.map {
  case ((borough, hour), count) => s"$borough,$hour,$count"
}

result.saveAsTextFile("data/orders_by_borough_hour")